In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
import re
import pyodbc
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import json
import os
from selenium.webdriver.support import expected_conditions
from sc_header import quitar_modales, createDriver, get_categorias, get_all_urls, scrape_batch, scrape_url,get_hrefs

In [2]:
start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')
url_base = "https://oldnavy.gap.com"

In [3]:
ipath = f'OLDNAVY_LINKS{fecha}.xlsx'

dfLinks = pd.read_excel(ipath)

In [4]:
dfLinks = dfLinks[dfLinks['sexo']=='Women']

In [5]:
browser = createDriver()
browser.get(url_base)

In [6]:
quitar_modales(browser)

In [7]:
dfLinks.reset_index(drop=True,inplace=True)

In [8]:
df = get_hrefs(dfLinks,browser)

In [9]:
start_ = datetime.datetime.now()
items = []

batch_size = 5
url_chunks = [df[x:x+batch_size] for x in range(0, len(df), batch_size)]

for url_chunk in url_chunks:
    items.append(scrape_batch(url_chunk))
end_ = datetime.datetime.now()

In [10]:
len(items)

216

In [15]:
browser.quit()

In [11]:
import itertools
def aplanar_lista(lista):
    if type(lista[0][0]) != list:
        return lista
    else:
        lista = list(itertools.chain(*lista))
        return aplanar_lista(lista)

In [12]:
new_list = aplanar_lista(items)

df = pd.DataFrame(new_list)

In [13]:
len(df)

1076

In [14]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [505]:
if 1 == 2:
    
    #url_aux = items[0][0][0][-2]
                             
    response = requests.get(url_aux)
    soup = BeautifulSoup(response.content, "html.parser")
    try:
        time.sleep(.3)
        metadatos = json.loads(soup.find('script', type='application/ld+json').text)
    except:
        print(url_aux)

    id_producto = metadatos[0]['productID'] #--- ID PRODUCTO
    descripcion = metadatos[0]['name'] #--- DESCRIPCION

    F = []
    pos_aux = 0

    for item in metadatos[0]['offers']:

        sku = item['itemOffered']['sku'] #SKU
        jpg = item['itemOffered']['image'] #JPG
        marca = item['seller']['name'] #MARCA
        color = item['itemOffered']['color'] #COLOR
        precio_dto = item['price'] #PRECIO

        precio = precio_dto
        condicion = item['itemCondition']#CONDICION : NEW/ Y NOSE
        disponibilidad = item['availability'] #DISPONIBILIDAD : INSTOCK/OUTSTOCK

        F.append([pos_aux,
                id_producto,
                descripcion,
                precio,
                sku,
                jpg,
                marca,
                color,
                precio_dto,
                condicion,
                disponibilidad,
                url_aux])


In [526]:
df[6].apply(lambda x: x[-4:])

0       0000
1       0001
2       0002
3       0003
4       0004
5       0005
6       0104
7       0204
8       0304
9       0404
10      0504
11      4002
12      4003
13      4004
14      4005
15      4006
16      0000
17      0001
18      0002
19      0003
20      0004
21      0005
22      0001
23      0002
24      0003
25      0004
26      0005
27      0000
28      0000
29      0001
        ... 
2126    4005
2127    4006
2128    4002
2129    4003
2130    4004
2131    4005
2132    4006
2133    4003
2134    4004
2135    4005
2136    4006
2137    4002
2138    0000
2139    0001
2140    0002
2141    0003
2142    0004
2143    0005
2144    0000
2145    0001
2146    0002
2147    0003
2148    0004
2149    0005
2150    0000
2151    0001
2152    0002
2153    0003
2154    0004
2155    0005
Name: 6, Length: 2156, dtype: object

In [550]:
master_talles = pd.DataFrame({'codigo_talle':['0000','0001','0002','0003','0004','0005','0104','0204','0304','0404',
                                              '0504','1004','4002','4003','4004','4005','4006','4007'],
                              'nombre_talle':['REGULAR XS','REGULAR S','REGULAR M','REGULAR L','REGULAR XL','REGULAR XXL',
                                              'TALL S','TALL M','TALL L','TALL XL','TALL XXL','TALL XS','PETITE XS',
                                              'PETITE S','PETITE M','PETITE L','PETITE XL','PETITE XXL']
            })

In [557]:
df.rename(columns={0:'pos',
                  1:'id_producto',
                  2:'descripcion',
                  3:'tipo',
                  4:'precio',
                  5:'sexo',
                  6:'sku',
                  7:'url_imagen',
                  8:'marca',
                  9:'color',
                  10:'precio_dto',
                  11:'condicion',
                  12:'disponibilidad',
                  13:'url_producto',
                  14:'pagina_scraper'},inplace=True)

In [559]:
master_talles

,codigo_talle,nombre_talle
0,0000,REGULAR XS
1,0001,REGULAR S
2,0002,REGULAR M
3,0003,REGULAR L
4,0004,REGULAR XL
5,0005,REGULAR XXL
6,0104,TALL S
7,0204,TALL M
8,0304,TALL L
9,0404,TALL XL


In [562]:
df['col_aux'] = df['sku'].apply(lambda x:x[-4:])

In [568]:
df = pd.merge(df,master_talles,left_on='col_aux',right_on='codigo_talle',how='inner')

In [399]:
url_aux='https://oldnavy.gap.com/browse/product.do?pid=481031002&cid=1127604&pcid=72091&vid=1&grid=pds_204_678_2#pdp-page-content'
response = requests.get(url_aux)
soup = BeautifulSoup(response.content, "html.parser")


In [400]:
metadatos = json.loads(soup.find('script', type='application/ld+json').text)

In [393]:
url_chunk 

,href,pagina_scraper,tipo,sexo,pos
310,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,50
311,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,51
312,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,52
313,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,53
314,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,54


In [359]:
try:
    df_save = pd.DataFrame(items)
    df_save.to_excel('zaful3.xlsx')
except:
    print("Error al guardar")    

,href,pagina_scraper,tipo,sexo,pos
0,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,0
1,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,1
2,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,2
3,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,3
4,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,4


,href,pagina_scraper,tipo,sexo,pos
0,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,0
1,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,1
2,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,2
3,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,3
4,https://oldnavy.gap.com/browse/product.do?pid=...,https://oldnavy.gap.com/browse/category.do?cid...,Tops,Women,4


4454

In [ ]:
lista_productos = []
metadatos = json.loads(soup.find('script', type='application/ld+json').text)

id_producto = metadatos[0]['productID'] #--- ID PRODUCTO
descripcion = metadatos[0]['name'] #--- DESCRIPCION
## tipologia = soup.find(class_='catnav--item catnav--item-selected').text #--- TIPOLOGIA
precio = soup.find(class_='product-price__strike').text #--- PRECIO
for item in metadatos[0]['offers']:
    
    sku = item['itemOffered']['sku'], #SKU
    jpg = item['itemOffered']['image'], #JPG
    marca = item['seller']['name'], #MARCA
    color = item['itemOffered']['color'], #COLOR
    precio_dto = item['price'], #PRECIO
    condicion = item['itemCondition'],#CONDICION : NEW/ Y NOSE
    disponibilidad = item['availability'] #DISPONIBILIDAD : INSTOCK/OUTSTOCK
    
    lista_productos.append([id_producto,
                            descripcion,
                            tipologia,
                            precio,
                            sku,
                            jpg,
                            marca,
                            color,
                            precio_dto,
                            condicion,
                            disponibilidad,
                           href])
    #FALTA HREF, FALTA POSICION, FALTA PAGINA SCRAPER

In [172]:
print(browser.current_url)

https://oldnavy.gap.com/browse/category.do?cid=50058&mlink=5151,,flyout_women_ShopbyCategory_Pajamas_Loungewear&clink=15682852#style=1043322,1097496,87786,69680,1113411,1144684,1097250,1143093,1093064,1127664


In [171]:
browser.save_screenshot('asd.png')

True

In [150]:
element_style = WebDriverWait(browser, 50).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".tabs--object.tabs--header-object.facet-style")))

In [162]:
element_style.click()

In [163]:
WebDriverWait(element_style, 50).until(EC.presence_of_all_elements_located((By.TAG_NAME, "input")))[0].click()

In [168]:
excepts

[['https://oldnavy.gap.com/browse/category.do?cid=1051876&mlink=5151,,flyout_women_ShopbyCategory_Jumpsuits&clink=15682852',
  'Jumpsuits'],
 ['https://oldnavy.gap.com/browse/category.do?cid=79586&mlink=5151,,flyout_women_ShopbyCategory_Skirts_&clink=15682852',
  'Skirts ']]

ERROR DE DIVISION


In [132]:
try:
    element = WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, "t.facet-style")))
    element.find_elements_by_tag_name('div')
except TimeoutException:
    print("TIMEOUT ERROR")
except NoSuchElementException:
    print("Elemento no se encuentra en la página, revisar el html")
except 
    
    
    

TIMEOUT ERROR


In [111]:
element_style.click()
browser.save_screenshot('asd.png')

True

In [110]:
try:
    element_style.find_elements_by_tag_name('input')[index_categoria].click()
except IndexError:
    print('ERROR AL CLICKEAR, FUERA DE RANGO')
    


['Pajama Sets',
 'Pajama Pants',
 'Pajama Shorts',
 'Tops',
 'Thermal',
 'Loungewear Bottoms',
 'One Piece',
 'Robes',
 'Slippers',
 'Accessories',
 'Clearance']

In [108]:
index_categoria

10

In [93]:
cantidad_categorias

['sin categoria']

In [94]:
browser.get("https://oldnavy.gap.com/browse/product.do?pid=412063032&cid=72091&pcid=72091&vid=1&grid=pds_0_615_1&cpos=0&cexp=1414&cid=CategoryIDs%3D72091&cvar=10353&ctype=Listing&cpid=res20010711851793332887717#pdp-page-content")

In [95]:
soup= BeautifulSoup(browser.page_source)

In [41]:
flag_checkbox

False

In [43]:
browser.save_screenshot('asd.png')

True

314

In [16]:
print(browser.current_url)

https://oldnavy.gap.com/browse/category.do?cid=72091&mlink=5151,,flyout_women_ShopbyCategory_Tops&clink=15682852#style=1127224,1036992,1127604


In [17]:
browser.save_screenshot('asd.png')

True

In [16]:
browser.refresh()

In [24]:
browser.get(row['url'])

In [42]:
browser.save_screenshot('asd.png')

True

In [58]:
browser.save_screenshot('asd.png')

True

In [234]:
browser.find_element_by_class_name('icon-chevron-right').click()

In [274]:
len(HREFS)

144

In [267]:
for i in HREFS_UNIQUE:
    print(i)

https://oldnavy.gap.com/browse/product.do?pid=503591042&cid=72091&pcid=72091&vid=1&grid=pds_94_584_1&cpos=94&cexp=1414&cid=CategoryIDs%3D72091&cvar=10356&ctype=Listing&cpid=res20010610510586198191431
https://oldnavy.gap.com/browse/product.do?pid=503949012&cid=72091&pcid=72091&vid=1&grid=pds_131_584_1&cpos=141&cexp=1414&cid=CategoryIDs%3D72091&ctype=Listing&cpid=res20010610510586198191431
https://oldnavy.gap.com/browse/product.do?pid=409748022&cid=72091&pcid=72091&vid=1&grid=pds_10_584_1&cpos=10&cexp=1414&cid=CategoryIDs%3D72091&cvar=10356&ctype=Listing&cpid=res20010610510586198191431
https://oldnavy.gap.com/browse/product.do?pid=449786002&cid=72091&pcid=72091&vid=1&grid=pds_137_584_1&cpos=148&cexp=1414&cid=CategoryIDs%3D72091&ctype=Listing&cpid=res20010610510586198191431
https://oldnavy.gap.com/browse/product.do?pid=773144002&cid=72091&pcid=72091&vid=1&grid=pds_85_584_1&cpos=85&cexp=1414&cid=CategoryIDs%3D72091&cvar=10356&ctype=Listing&cpid=res20010610510586198191431
https://oldnavy.ga

In [240]:
browser.find_elements_by_class_name('product-card__link')[-1].get_attribute('href')

'https://oldnavy.gap.com/browse/product.do?pid=380894032&cid=1127604&pcid=72091&vid=1&grid=pds_299_610_2'

In [233]:
browser.find_elements_by_class_name('product-card__link')[-1].get_attribute('href')

'https://oldnavy.gap.com/browse/product.do?pid=484822032&cid=1036992&pcid=72091&vid=1&grid=pds_299_624_1'

In [262]:
len(browser.find_elements_by_class_name('product-card__link'))

288

In [244]:
browser.execute_script('history.back()')

In [238]:
browser.save_screenshot('asd.png')

True

In [169]:
browser = createDriver()

In [170]:
browser.get(browser.current_url)

In [ ]:
    #"cantidad de colores"
    time.sleep(.8)
    try:

        aux_col = [i for i in browser.find_elements_by_class_name('tabs--object') if i.text.upper() == 'COLOR'][0]            
        cantidad_colores = [i.get_attribute('name') for i in aux_col.find_element_by_class_name('accordion--content--inner').find_elements_by_tag_name('input')]
    except:
        excepts.append(browser.current_url)


    #LOOP CANTIDAD DE COLORES
    for index_color,nombre_color in enumerate(cantidad_colores):

        #CLICK EN EL COLOR DEL INDICE
#            browser.implicitly_wait(30)
        time.sleep(.8)
        try:

            element_color = browser.find_element_by_css_selector('.tabs--object.tabs--header-object.facet-color')
            element_color.click()
            element_color.find_elements_by_tag_name('input')[index_color].click()
            element_color.click()
        except:
            pass

        if '0 Items in the product grid' == browser.execute_script('''var x = document.getElementsByClassName('category__item-count')[0].textContent;return x;'''):

            #QUITAR FILTRO DE COLOR Y SEGUIR EL SCRAPEO
            browser.find_elements_by_css_selector('.tag--icon.icon-tag-x')[len(cantidad_categorias)+1].click()
            
            continue

        #NOMBRE COLOR
        #CANTIDAD DE PRODUCTOS DESPUÉS DE FILTRAR
        time.sleep(.8)
        try:
            cant_productos = len(browser.find_element_by_css_selector('.product-card-grid__root.grid.ism-root').find_elements_by_css_selector('.product-card-grid__item-1-2.product-card-grid__item-lg-1-3.product-card-grid__item-xl-1-3.product-card-grid__item-1280-1-4'))
        except:
            
            continue

        soup = BeautifulSoup(browser.page_source)

        html_plano = [i for i in soup.find_all(class_='product-card-grid__root grid ism-root')][0]

        for index_p,html in enumerate(html_plano):

            try:

                lista_productos.append([index_p,
                                        html.a['id'],
                                        html.a.img['alt'],
                                        row['sexo'],
                                        row['tipo'],
                                        [i.text for i in html.find_all(class_='product-card-price')],
                                        html.a.img['src'],
                                        html.a['href'],
                                        nombre_categoria,
                                        nombre_color])
            except:
                EX.append(html)

        try:

            #BORRAR FILTRO COLOR
            browser.find_elements_by_css_selector('.tag--icon.icon-tag-x')[len(cantidad_categorias)+1].click()
            
        except:
            print("no borro filtro color")
            try:
                element_color = browser.find_element_by_css_selector('.tabs--object.tabs--header-object.facet-color')
                element_color.click()
                element_color.find_elements_by_tag_name('input')[index_color].click()
                element_color.click()
            except:
                print("No borro filtro color en 2 oportunidades")
                raise

## DEBUG

In [157]:
#DEBUG
lista_productos = []
excepts = []
EX = []
for index,row in dfLinks.iterrows():
    try:
        browser.delete_all_cookies()
        browser.get(row['url'])
    except:
        browser.quit()
        time.sleep(1)
        browser = createDriver()
        
        continue
    
    #"cantidad categorias"
    if '0 Items in the product grid' == browser.execute_script('''var x = document.getElementsByClassName('category__item-count')[0].textContent;return x;'''):
        
        continue
    
    try:
        
        time.sleep(2)
        #DUDA
        category = [i for i in browser.find_elements_by_class_name('tabs--object') if i.text.upper() == 'CATEGORY'][0]
        cantidad_categorias = [i.get_attribute('name') for i in category.find_element_by_class_name('accordion--content--inner').find_elements_by_tag_name('input')]
    except:
        cantidad_categorias = ['sin categoria']
        excepts.append([row['url'],
                       row['tipo']])
        
        continue

    #FOR DE CANTIDAD DE CATEGORIAS
    for index_categoria,nombre_categoria in enumerate(cantidad_categorias):

        #CLICK EN LA CATEGORIA DEL INDICE

        element_style = browser.find_element_by_css_selector('.tabs--object.tabs--header-object.facet-style')
        element_style.click()
        element_style.find_elements_by_tag_name('input')[index_categoria].click()
        element_style.click()

            

    #"cantidad de colores"
    time.sleep(.8)
    try:

        aux_col = [i for i in browser.find_elements_by_class_name('tabs--object') if i.text.upper() == 'COLOR'][0]            
        cantidad_colores = [i.get_attribute('name') for i in aux_col.find_element_by_class_name('accordion--content--inner').find_elements_by_tag_name('input')]
    except:
        excepts.append(browser.current_url)


    #LOOP CANTIDAD DE COLORES
    for index_color,nombre_color in enumerate(cantidad_colores):

        #CLICK EN EL COLOR DEL INDICE
#            browser.implicitly_wait(30)
        time.sleep(.8)
        try:

            element_color = browser.find_element_by_css_selector('.tabs--object.tabs--header-object.facet-color')
            element_color.click()
            element_color.find_elements_by_tag_name('input')[index_color].click()
            element_color.click()
        except:
            pass

        if '0 Items in the product grid' == browser.execute_script('''var x = document.getElementsByClassName('category__item-count')[0].textContent;return x;'''):

            #QUITAR FILTRO DE COLOR Y SEGUIR EL SCRAPEO
            browser.find_elements_by_css_selector('.tag--icon.icon-tag-x')[len(cantidad_categorias)+1].click()
            
            continue

        #NOMBRE COLOR
        #CANTIDAD DE PRODUCTOS DESPUÉS DE FILTRAR
        time.sleep(.8)
        try:
            cant_productos = len(browser.find_element_by_css_selector('.product-card-grid__root.grid.ism-root').find_elements_by_css_selector('.product-card-grid__item-1-2.product-card-grid__item-lg-1-3.product-card-grid__item-xl-1-3.product-card-grid__item-1280-1-4'))
        except:
            
            continue

        soup = BeautifulSoup(browser.page_source)

        html_plano = [i for i in soup.find_all(class_='product-card-grid__root grid ism-root')][0]

        for index_p,html in enumerate(html_plano):

            try:

                lista_productos.append([index_p,
                                        html.a['id'],
                                        html.a.img['alt'],
                                        row['sexo'],
                                        row['tipo'],
                                        [i.text for i in html.find_all(class_='product-card-price')],
                                        html.a.img['src'],
                                        html.a['href'],
                                        nombre_categoria,
                                        nombre_color])
            except:
                EX.append(html)

        try:

            #BORRAR FILTRO COLOR
            browser.find_elements_by_css_selector('.tag--icon.icon-tag-x')[len(cantidad_categorias)+1].click()
            
        except:
            print("no borro filtro color")
            try:
                element_color = browser.find_element_by_css_selector('.tabs--object.tabs--header-object.facet-color')
                element_color.click()
                element_color.find_elements_by_tag_name('input')[index_color].click()
                element_color.click()
            except:
                print("No borro filtro color en 2 oportunidades")
                raise

no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
no borro filtro color
No borro filtro color en 2 oportunidades


IndexError: list index out of range

In [27]:
df = pd.DataFrame(lista_productos)

In [28]:
df.rename(columns={0:'sku',
                  1:'descripcion',
                  2:'categoria',
                  3:'tipo',
                  4:'prices',
                  5:'img',
                  6:'url',
                  7:'sub_categoria',
                  8:'color'},inplace=True)

In [29]:
df['prices'] = df['prices'].apply(lambda x:  x[0])

df['precio_dto'] = df['prices'].apply(lambda x : x.split('$')[2] if len(x.split('$')) == 3 else x.split('$')[1])

df['precio_original'] = df['prices'].apply(lambda x : x.split('$')[1] if len(x.split('$')) == 3 else x.split('$')[1])

In [30]:
df = df.drop_duplicates()

In [31]:
browser.quit()

In [34]:
df.to_excel(f'./Salida/OLDNAVY_WOMEN{fecha}.xlsx')

In [32]:
%hist

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
#pdb.set_trace()
from ipywidgets import IntProgress
from IPython.display import display
import os
from selenium.webdriver.support import expected_conditions
from IPython.display import clear_output
TIEMPO_DE_ESPERA = 5.5
start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

chrome_options = Options()

In [652]:
len(df)

235148

In [622]:
so = BeautifulSoup('''<div class="clearfix">
<a href="/tolink1">Link1</a>
<a href="/tolink2">Link2</a>
</div>''')




In [651]:
so = BeautifulSoup('''<div class="clearfix">
<a href="/tolink1">Link1</a>
<a href="/tolink2">Link2</a>
</div>''')



for i in so.find_all(class_='clearfix'):
    for m in i:
        try:
             print(m['href'])
        except:
            pass
    

        
    

/tolink1
/tolink2


/tolink1
/tolink2


In [646]:
m

'\n'